In [1]:
import pandas as pd
import numpy as np
#importing NLP libraries
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
#Preprocessing libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
#Modeling
from sklearn.svm import SVC
from sklearn.metrics import f1_score

In [3]:
open('spam.csv', encoding="ISO-8859-1")

<_io.TextIOWrapper name='spam.csv' mode='r' encoding='ISO-8859-1'>

In [11]:
data = pd.read_csv('spam.csv',encoding='ISO-8859-1')

In [12]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [13]:
data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [14]:
y = data['v1']
X = data.drop('v1', axis=1)

In [15]:
X

,v2
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro..."
...,...
5567,This is the 2nd time we have tried 2 contact u...
5568,Will Ì_ b going to esplanade fr home?
5569,"Pity, * was in mood for that. So...any other s..."
5570,The guy did some bitching but I acted like i'd...


In [16]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
pd.DataFrame(y)

,0
0,0
1,0
2,1
3,0
4,0
...,...
5567,1
5568,0
5569,0
5570,0


In [17]:
class_mapping = {i : v for i,v in enumerate(encoder.classes_)}
class_mapping

{0: 'ham', 1: 'spam'}

In [18]:
def processEmail(contents):
    ps = PorterStemmer()
    
 
    contents = contents.lower()
    
    contents = re.sub(r'<[^<>]+>', ' ', contents)
    
    contents = re.sub(r'[0-9]+', 'number', contents)
    
    contents = re.sub(r'(http|https)://[^/s]*', 'httpaddr', contents)
    contetns = re.sub(r'[^/s]+@[^/s]+', 'emailaddr', contents)
    
    contents = re.sub(r'[$]+', 'dollar', contents)
    
    words = word_tokenize(contents)
    
    for i in range(len(words)):
        words[i] = re.sub(r'[^a-zA-Z0-9]', '', words[i])
        words[i] = ps.stem(words[i])
        
    words = [word for word in words if len(word)>=1]
    return words

In [19]:
def getVocabulary(emails, vocab_length):
    vocabulary= dict()
    
    for i in range(len(emails)):
        emails[i] = processEmail(emails[i])
        for word in emails[i]:
            if word in vocabulary.keys():
                vocabulary[word]+=1
            else:
                vocabulary[word]=1
    
    vocabulary = sorted(vocabulary.items(), key = lambda x : x[1], reverse=True)
    vocabulary = list(map(lambda x : x[0], vocabulary[:vocab_length]))
    vocabulary = {i:v for i,v in enumerate(vocabulary)}
    
    return vocabulary

In [20]:
data['v2'].tolist()

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though",
 "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",
 'Even my brother is not like to speak with me. They treat me like aids patent.',
 "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
 'WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.',
 'Had y

In [21]:
def getkey(dictionay, val):
    for key, value in dictionay.items():
        if value==val:
            return key

In [22]:
def getIndices(email, vocabulary):
    word_indices = set()
    
    for word in email:
        if word in vocabulary.values():
            word_indices.add(getkey(vocabulary, word))
    
    return word_indices

In [23]:
def getFeatureVector(word_indices, vocab_length):
    feature_vec = np.zeros(vocab_length)
    
    for i in word_indices:
        feature_vec[i]=1
    
    return feature_vec

In [24]:
vocab_length = 2000

In [27]:
import nltk
nltk.download('punkt')
vocabulary = getVocabulary(data['v2'].to_list(),vocab_length)
emails = data['v2'].to_list()
emails = list(map(lambda x : processEmail(x), emails))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [28]:
emails

[['go',
  'until',
  'jurong',
  'point',
  'crazi',
  'avail',
  'onli',
  'in',
  'bugi',
  'n',
  'great',
  'world',
  'la',
  'e',
  'buffet',
  'cine',
  'there',
  'got',
  'amor',
  'wat'],
 ['ok', 'lar', 'joke', 'wif', 'u', 'oni'],
 ['free',
  'entri',
  'in',
  'number',
  'a',
  'wkli',
  'comp',
  'to',
  'win',
  'fa',
  'cup',
  'final',
  'tkt',
  'numberst',
  'may',
  'number',
  'text',
  'fa',
  'to',
  'number',
  'to',
  'receiv',
  'entri',
  'question',
  'std',
  'txt',
  'rate',
  't',
  'c',
  's',
  'appli',
  'numberovernumb',
  's'],
 ['u', 'dun', 'say', 'so', 'earli', 'hor', 'u', 'c', 'alreadi', 'then', 'say'],
 ['nah',
  'i',
  'do',
  'nt',
  'think',
  'he',
  'goe',
  'to',
  'usf',
  'he',
  'live',
  'around',
  'here',
  'though'],
 ['freemsg',
  'hey',
  'there',
  'darl',
  'it',
  's',
  'been',
  'number',
  'week',
  's',
  'now',
  'and',
  'no',
  'word',
  'back',
  'i',
  'd',
  'like',
  'some',
  'fun',
  'you',
  'up',
  'for',
  'it',
 

In [36]:
X = list(map(lambda x : getFeatureVector(getIndices(x, vocabulary), vocab_length), emails))

In [39]:
X

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 1., 1., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([1., 0., 1., ..., 0., 0., 0.]),
 array([1., 1., 1., ..., 0., 0., 0.]),
 array([0., 0., 1., ..., 0., 0., 0.]),
 array([0., 1., 1., ..., 0., 0., 0.]),
 array([0., 1., 1., ..., 0., 0., 0.]),
 array([0., 1., 1., ..., 0., 0., 0.]),
 array([1., 0., 1., ..., 0., 0., 0.]),
 array([0., 1., 1., ..., 0., 0., 0.]),
 array([0., 1., 1., ..., 0., 0., 0.]),
 array([1., 0., 1., ..., 0., 0., 0.]),
 array([1., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 1., ..., 0., 0., 0.]),
 array([1., 0., 0., ..., 0., 0., 0.]),
 array([1., 1., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 1., 1., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 1., 1., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([1., 1., 0., ..., 0., 0., 0.]),
 array([1., 0., 0., ..., 0., 0., 0.]),
 array([1., 0., 1., ..., 

In [41]:
X = pd.DataFrame(np.array(X).astype(np.int16))

In [42]:
X

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,1,0,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5568,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5570,1,0,1,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [44]:
model = SVC()
model.fit(X_train, y_train)

SVC()

In [45]:
np.sum(y)/len(y)

0.13406317300789664

In [46]:
model.score(X_test, y_test)

0.9910313901345291

In [47]:
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

0.9635036496350364